In [145]:
import pandas as pd
import re
from selenium import webdriver as wd
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException

### 변환할 데이터 불러오기 

In [66]:
col_names = ['name','number','address','add1','add2']

In [119]:
df = pd.read_csv('./geoData/japan_encephalitis2.csv', encoding='utf8', names=col_names, usecols=[0,1,2,3,4])

In [122]:
df.head(2)

,name,number,address,add1,add2,lat,lng
0,365열린의원,041-549-3650,충청남도 아산시 용화로9번길 4-22,(용화동,365건물) 2층,0.0,0.0
1,가람내과의원,041-544-1101,충청남도 아산시 배방읍 북수로 106,2층,NaN,0.0,0.0


In [121]:
df['lat'] = 0.0
df['lng'] = 0.0

In [56]:
df.iloc[:3]

,병/의원명,전화번호,주소,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,&#40;사&#41;인구보건복지협회경기도지회가족보건의원,031-256-4644,경기도 수원시 팔달구 고화로14번길 11,(매산로3가),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,&#40;사&#41;한국건강관리협회건강증진의원,031-250-5800,경기도 수원시 장안구 수성로 330,(영화동),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,&#40;의&#41;영문의료재단 다보스병원,031-8021-2114,경기도 용인시 처인구 백옥대로1082번길 18,(김량장동) 1082-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
df.head(5)

,name,number,address,add1,add2,lat
0,365열린의원,041-549-3650,충청남도 아산시 용화로9번길 4-22,(용화동,365건물) 2층,0
1,가람내과의원,041-544-1101,충청남도 아산시 배방읍 북수로 106,2층,,0
2,강경삼성의원,041-745-7119,충청남도 논산시 강경읍 계백로 152-1 대흥리 23-4,,,0
3,강이비인후과의원,041-584-7774,충청남도 천안시 서북구 직산읍 봉주로 41 삼은리 317,,,0
4,경희의원,041-546-4051,충청남도 아산시 온천대로 1474,(온천동),,0


### Fill NaN to space

### Drop column and Rename

In [123]:
df.fillna('', inplace=True)

In [124]:
df.tail(3)

,name,number,address,add1,add2,lat,lng
6305,히즈메디소아청소년과의원,031-985-1365,경기도 김포시 중봉로 1,(감정동,신안3차) 3층,0.0,0.0
6306,힐락소아청소년과의원,032-624-7033,경기도 부천시 송내대로 239,(상동,부천터미널소풍) 3R51호,0.0,0.0
6307,힐메디의원,031-985-7572,경기도 김포시 고촌읍 수기로 67-39,(수기마을힐스테이트2단지 상가동) 201호,,0.0,0.0


### Crawling to fill lat, lng

In [26]:
driver = wd.Chrome(executable_path='chromedriver.exe')

In [130]:
def getLatLng(addr):
    driver.get('http://f.goodkiss.co.kr/naver/naverMap.html')
    driver.implicitly_wait(1)
    driver.find_element_by_xpath('/html/body/form/span/input[1]').send_keys(addr)
    driver.find_element_by_xpath('/html/body/form/span/input[2]').click()
    driver.implicitly_wait(1)
    soup = bs(driver.page_source, 'lxml')
    lat = float(soup.findAll('td')[4].text)
    lng = float(soup.findAll('td')[5].text)
    return lat, lng 

In [143]:
for i in range(543,len(df)): #df
    try:
        lat, lng = getLatLng(df.address[i])
    except IndexError:
        lat, lng = 0, 0
    except NoSuchElementException:
        lat, lng = 0, 0
    df.lat[i] = lat
    df.lng[i] = lng
    if i % 500 == 0:
        print(i, end=' ')

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


NameError: name 'NoSuchElementException' is not defined

In [142]:
df.iloc[530:550]

,name,number,address,add1,add2,lat,lng
530,덕양가정의학과의원,032-872-8275,인천광역시 미추홀구 경인로 231,(도화동),,126.668227,37.464448
531,도담소아청소년과의원,032-464-1924,인천광역시 남동구 독점로30번길 1,(구월동,에이스프라자) 2층 일부호,126.711122,37.452427
532,도담이비인후과의원,032-814-7582,인천광역시 연수구 원인재로 312,(연수동,대림상가) 3층 304호,126.696044,37.422317
533,도화연세소아과의원,032-863-7004,인천광역시 미추홀구 석정로 300,(도화동),,126.668798,37.467880
534,동부제일의원,032-761-3171,인천광역시 동구 화수로 18,(송현동,동부센트레빌 상가) 303호,126.635627,37.481208
535,동인천내과의원,032-765-5676,인천광역시 동구 송현로 1,(송현동,송현프라자) 104호,126.633319,37.477038
536,두리산부인과두리내과의원,032-545-7030,인천광역시 계양구 계양대로 83,(작전동) 2층,,126.722212,37.531242
537,두림메디컬의원,032-361-7765,인천광역시 부평구 부흥로 408,(부개동) 2층,,126.737312,37.497460
538,드림 소아청소년과의원,032-934-1175,인천광역시 강화군 강화읍 동문안길 9-1 2층 드림소아청소년과의원,,,0.000000,0.000000
539,딸기소아청소년과의원,032-435-4275,인천광역시 남동구 앵고개로847번길 64,(논현동,골든프라자) 3층 302호,126.725031,37.398750


In [ ]:
# 혹시 모르니 복사해놓기
safe = df.copy()

In [45]:
# 주소 풀네임 
def address_fullName(df):
    pattern = re.compile('[^a-zA-Z0-9가-힣]|NaN')
    for i in range(len(df)):
        add1 = re.sub(pattern, '', df.add1[i])
        add2 = re.sub(pattern, '', df.add2[i])
        address = df.address[i]
        df.address[i] = ' '.join([address,add1,add2])
    df.drop('add1', axis=1, inplace=True)
    df.drop('add2', axis=1, inplace=True)
    return df

In [46]:
df = address_fullName(df)

In [47]:
# mongoDB에 한번에 넣을 수 있게 만든 column
df['total'] = df.apply(lambda x: dict(name=x['name'],
                                    number=x['number'],
                                    address=x['address'],
                                    lat=x['lat'],
                                    lng=x['lng']),axis=1)

### MongoDB

In [49]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
disease_db = client.get_database('disease_db')

In [50]:
japanEncephalitis_collection = disease_db.get_collection('japan_encephalitis2')
japanEncephalitis_collection.insert_many([
    df.total[i] for i in range(len(df))
])

![image](https://user-images.githubusercontent.com/33097467/44586971-76ad7600-a7ec-11e8-9f11-0618a4f3d963.png)
